In [145]:
import pandas as pd
import os

In [146]:
APPROACH = "spinfer"

In [147]:
df = pd.read_csv(f"computed_{APPROACH}.csv")
df["id"] = [x for x in range(len(df))]
df.head(3)

,root,cocci_filename,c_filename,pred,is_cocci_valid,is_applied,diff_truth,diff_pred,diff_5,precision,recall,cocci_fullpath,rank_score,cocci,id
0,./EXP0-7,sp_out.final.spinfer.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrf...,out,1,True,"- if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &fs_i...",NaN,"--- initial\n+++ final\n@@ -1,10 +1,10 @@\n vo...",0.0,0.0,./EXP0-7/sp_out.final.spinfer.cocci,0.333333,"@@\nexpression E0, E1;\n@@\n- E0->objectid = E...",0
1,./EXP0-7,sp_out.final.spinfer.cocci,1533533124_2018-08-06_4fd786e6c3d6_super_btrfs...,out,1,True,- buf->f_fsid.val[0] ^= BTRFS_I(d_inode(dentr...,NaN,"--- initial\n+++ final\n@@ -65,9 +65,9 @@\n ...",0.0,0.0,./EXP0-7/sp_out.final.spinfer.cocci,0.333333,"@@\nexpression E0, E1;\n@@\n- E0->objectid = E...",1
2,./EXP0-7,sp_out.final.spinfer.cocci,1533533124_2018-08-06_4fd786e6c3d6_ctree_add_r...,out,1,True,- if (root->objectid == BTRFS_EXTENT_TREE...,NaN,"--- initial\n+++ final\n@@ -5,11 +5,11 @@\n ...",0.0,0.0,./EXP0-7/sp_out.final.spinfer.cocci,0.333333,"@@\nexpression E0, E1;\n@@\n- E0->objectid = E...",2


# Analyze valid semantic patches that fail to transform unseen C codes (0 Precision and 0 Recall)

In [148]:
mask1 = df["precision"] == 0
mask3 = df["recall"] == 0
mask2 = df["is_cocci_valid"] == 1
valid_sp_fail_transform = df[(mask1) & (mask2) & (mask3)]
# temp_df = temp_df.sample(n=50, random_state=123)
# temp_df.sort_values(["root", "c_filename"], inplace=True)
count_fail = len(valid_sp_fail_transform)

In [171]:
valid_sp_fail_transform[["root", "c_filename"]].to_csv("temp.csv", index=False)

In [172]:
count = 1
for idx, row in valid_sp_fail_transform.iterrows():
    print(count)
    count += 1
    # print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    debug_filepath = os.path.join(row["root"], row["c_filename"] + f".out.{APPROACH}.res.txt")
    with open(debug_filepath, "r") as f:
        debug = f.read()


    print(debug)
    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


1
init_defs_builtins: /usr/bin/../lib/coccinelle/standard.h
-----------------------------------------------------------------------
processing semantic patch file: ./EXP0-7/sp_out.final.spinfer.cocci
with isos from: /usr/bin/../lib/coccinelle/standard.iso
-----------------------------------------------------------------------
@@
expression E0, E1;
@@
- E0->objectid = E1; 
+ E0->root_key.objectid = E1; 
// Infered from: (./EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_disk-io___setup_root.{c.sanitized.c,res.c.sanitized.res.c}: __setup_root)
// Recall: 0.50, Precision: 1.00, Matching recall: 1.00

// ---------------------------------------------
@@
expression E0, E1;
@@
- btrfs_qgroup_free_refroot(BTRFS_I(E0)->root->fs_info,                           BTRFS_I(E0)->root->objectid, E1.bytes_changed,                           BTRFS_QGROUP_RSV_DATA); 
+ btrfs_qgroup_free_refroot(BTRFS_I(E0)->root->fs_info,                           BTRFS_I(E0)->root->root_key.objectid,                           E

In [151]:
count = 1
for idx, row in valid_sp_fail_transform.iterrows():
    print(count)
    count += 1
    print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


1
--- initial
+++ final
@@ -1,10 +1,10 @@
 void btrfs_qgroup_free_meta_all_pertrans(struct btrfs_root *root)
 {
   struct btrfs_fs_info *fs_info = root->fs_info;
-  if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &fs_info->flags) || !is_fstree(root->objectid))
+  if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &fs_info->flags) || !is_fstree(root->root_key.objectid))
     return;
 
   trace_qgroup_meta_free_all_pertrans(root);
 
-  btrfs_qgroup_free_refroot(fs_info, root->objectid, (u64)-1, BTRFS_QGROUP_RSV_META_PERTRANS);
+  btrfs_qgroup_free_refroot(fs_info, root->root_key.objectid, (u64)-1, BTRFS_QGROUP_RSV_META_PERTRANS);
 }
#############################################################
#############################################################
#############################################################
2
--- initial
+++ final
@@ -65,9 +65,9 @@
   buf->f_namelen = BTRFS_NAME_LEN;
 
   buf->f_fsid.val[0] = be32_to_cpu(fsid[0]) ^ be32_to_cpu(fsid[2]);
   buf->f_fsid.val[1] = be32_to_cpu(fsid[1]) ^ be

# Analyze valid semantic patches that fail to transform unseen C codes PARTIALLY (Precision > 0 and Recall > 0, but both still less than 1) 

In [173]:
mask1 = df["precision"] != 0
mask2 = df["precision"] != 1
mask3 = df["recall"] != 0
mask4 = df["recall"] != 1
mask5 = df["is_cocci_valid"] == 1
partial_sp = df[(mask1) & (mask2) & (mask3) & (mask4) & (mask5)]
# temp_df = temp_df.sample(n=50, random_state=123)
# temp_df.sort_values(["root", "c_filename"], inplace=True)
count_partial = len(partial_sp)

In [174]:
partial_sp[["root", "c_filename"]].to_csv("temp.csv", index=False)

In [175]:
count = 1
for idx, row in partial_sp.iterrows():
    print(count)
    count += 1
    # print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    debug_filepath = os.path.join(row["root"], row["c_filename"] + f".out.{APPROACH}.res.txt")
    with open(debug_filepath, "r") as f:
        debug = f.read()


    print(debug)
    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


1
init_defs_builtins: /usr/bin/../lib/coccinelle/standard.h
-----------------------------------------------------------------------
processing semantic patch file: ./dasd_smalloc/sp_out.final.spinfer.cocci
with isos from: /usr/bin/../lib/coccinelle/standard.iso
-----------------------------------------------------------------------
@@
identifier I1;
expression E0, E2, E3, E4;
@@
- E0 = dasd_smalloc_request(I1, E2, E3, E4); 
+ E0 = dasd_smalloc_request(DASD_ECKD_MAGIC, E2, E3, E4, NULL); 
// Infered from: (./dasd_smalloc/1528132059_2018-06-04_c5205f2ff2be_dasd_eckd_dasd_eckd_analysis_ccw.{c.sanitized.c,res.c.sanitized.res.c}: dasd_eckd_analysis_ccw)
// False positives: (./dasd_smalloc/1528132059_2018-06-04_c5205f2ff2be_dasd_diag_dasd_diag_build_cp.res.c.sanitized.res.c: dasd_diag_build_cp)
// Recall: 0.75, Precision: 0.75, Matching recall: 0.75

// ---------------------------------------------
// Final metrics (for the combined 1 rules):
// -- Edit Location --
// Recall: 1.00, Precision

In [176]:
count = 1
for idx, row in partial_sp.iterrows():
    print(count)
    count += 1
    print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


1
--- initial
+++ final
@@ -61,11 +61,11 @@
         count = 2 * blk_per_trk - first_rec;
       cplength += count;
       datasize += count * sizeof(struct LO_eckd_data);
     }
 
-  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, cplength, datasize, startdev);
+  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, cplength, datasize, startdev, blk_mq_rq_to_pdu(req));
   if (IS_ERR(cqr))
     return cqr;
   ccw = cqr->cpaddr;
 
   if (use_prefix)
#############################################################
#############################################################
#############################################################
2
--- initial
+++ final
@@ -38,11 +38,11 @@
   rq_for_each_segment(bv, req, iter) { ++ctidaw; }
   if (rq_data_dir(req) == WRITE)
     ctidaw += (last_trk - first_trk);
 
   itcw_size = itcw_calc_size(0, ctidaw, 0);
-  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, 0, itcw_size, startdev);
+  cqr = dasd_smalloc_request(DASD_ECKD_MAGIC, 0, itcw_size, startdev, blk_mq_rq_to_pd

# Analyze invalid semantic patches 

In [177]:
mask1 = df["precision"] == 0
mask3 = df["recall"] == 0
mask5 = df["is_cocci_valid"] != 1
invalid_sp = df[(mask1) & (mask3) & (mask5)]
# temp_df = temp_df.sample(n=50, random_state=123)
# temp_df.sort_values(["root", "c_filename"], inplace=True)
count_invalid = len(invalid_sp)

In [178]:
invalid_sp[["root", "c_filename"]].to_csv("temp.csv", index=False)

In [179]:
invalid_sp.root.value_counts()

root
./tcf_block_get-61    27
./snd_soc             20
./uartlite             3
Name: count, dtype: int64

# Analyze valid semantic patches

In [180]:
mask1 = df["precision"] == 1
mask3 = df["recall"] == 1
mask5 = df["is_cocci_valid"] == 1
valid_sp = df[(mask1) & (mask3) & (mask5)]
# temp_df = temp_df.sample(n=50, random_state=123)
# temp_df.sort_values(["root", "c_filename"], inplace=True)
count_correct = len(valid_sp)

In [181]:
(count_correct+count_fail+count_invalid+count_partial == len(df))

True

In [161]:
count_correct

25

In [162]:
count_fail

97

In [163]:
count_invalid

50

In [164]:
count_partial

5

In [165]:
72+32+65+5

174

In [166]:
temp = pd.concat([invalid_sp, partial_sp, valid_sp, valid_sp_fail_transform])

In [167]:
temp = df[~df.id.isin(temp.id)]

In [168]:
temp[["root", "c_filename"]].to_csv("temp.csv", index=False)

In [169]:
count = 1
for idx, row in temp.iterrows():
    print(count)
    count += 1
    # print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    debug_filepath = os.path.join(row["root"], row["c_filename"] + f".out.{APPROACH}.res.txt")
    with open(debug_filepath, "r") as f:
        debug = f.read()


    print(debug)
    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


In [170]:
count = 1
for idx, row in temp.iterrows():
    print(count)
    count += 1
    print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    
